In [ ]:

import numpy as np
from cvxpy import *

# Linear Program

In [75]:
# # p = Parameter(sign="positive", value=4)
# p = 4
# u = np.array([0.3, 0.25, 0.45])
# t = np.array([5.0,6.0,2.0]) #.reshape((i,1))
# # k = Parameter(sign="positive", value=3)
# k = 3
# # T = Parameter(sign="positive", value=10)
# T = 10

def mip_assign_slots(num_slots, utilities, throughputs, required_throughput):
    p = num_slots
#     print p
    k = len(utilities)
#     print k
    assert len(throughputs) == len(utilities)
    T = required_throughput
    X = Variable(k,p)
    r = Variable(k)
    q = Variable(k)
    maxs = []
    constr = [] 
    constr += [X >= 0, X <= 1]
    for i in range(k):
        constr += [q[i] == sum_entries(X[i,:])*t[i]/T - r[i]]
        maxs.append(Maximize(q[i]*u[i] - r[i]))

    for j in range(p): 
        constr += [sum_entries(X[:,j]) == 1]
   
    
    constr += [sum_entries(X) == p]
    constr += [q >= 0, q <= 1, r >= 0]
    prob = Problem(sum(maxs), constr)

    print(prob.solve())
    xs = np.zeros((k,p))
#     print X.value
    for i in range(k):
        for j in range(p):
            cur_x = X.value[i,j]
            if cur_x > 0.01:
                xs[i,j] = cur_x
    return xs
#     return X.value


In [ ]:
print r.value
print q.value

# Integer Program

In [ ]:
p = 4
u = np.array([0.3, 0.25, 0.45])
t = np.array([5.0,6.0,2.0]) #.reshape((i,1))
# k = Parameter(sign="positive", value=3)
k = 3
# T = Parameter(sign="positive", value=10)
T = 10

X = Int(k,p)
r = Variable(k)
q = Int(k)

maxs = []
constraints = []
for i in range(k):
#     obj = q[i]*u[i] - r[i]
#     constr = [q[i] == sum_entries(X[i,:])*t[i]/T - r[i]]
    maxs.append(Maximize(q[i]*u[i] - r[i]))
    constraints += [q[i] == sum_entries(X[i,:]) - r[i]]
    constraints += [sum_entries(X[i,:])*t[i] >= T*q[i]]



for j in range(p): 
    # should this be equality or lt_equals?
    constraints += [sum_entries(X[:,j]) == 1]
    

    
constraints += [sum_entries(X) == p]
constraints += [X <= 1]
constraints += [X >= 0]
constraints += [q <= 1]
constraints += [q >= 0]
constraints += [r >= 0]
prob = Problem(sum(maxs), constraints)


# print [c.is_dcp() for c in full_prob.constraints]

prob.solve()
xs = np.zeros((k,p))
for i in range(k):
    for j in range(p):
        cur_x = X.value[i,j]
        if cur_x > 0.0000001:
            xs[i,j] = cur_x
print xs
        
# res = [x if x > 0.0000001 else 0 for x in X.value]

In [39]:
import numpy as np
from cvxpy import *
def assign_slots(num_slots, utilities, throughputs, required_throughput):
# u = np.array([0.3, 0.25, 0.45])
# t = np.array([5.0,6.0,2.0]) #.reshape((i,1))
# # k = Parameter(sign="positive", value=3)
# k = 3
# # T = Parameter(sign="positive", value=10)
# T = 10)
    p = num_slots
    print p
    k = len(utilities)
    print k
    assert len(throughputs) == len(utilities)
    T = required_throughput
    X = Int(k,p)
    r = Variable(k)
    q = Int(k)
    maxs = []
    constraints = []
    for i in range(k):
        maxs.append(Maximize(q[i]*u[i] - r[i]))
        constraints += [q[i] == sum_entries(X[i,:]) - r[i]]
        constraints += [sum_entries(X[i,:])*t[i] >= T*q[i]]
    for j in range(p): 
        constraints += [sum_entries(X[:,j]) == 1]
    constraints += [sum_entries(X) == p]
    constraints += [X <= 1]
    constraints += [X >= 0]
    constraints += [q <= 1]
    constraints += [q >= 0]
    constraints += [r >= 0]
    prob = Problem(sum(maxs), constraints)
    print prob.solve()
#     print prob
    # filter small floats to 0
    xs = np.zeros((k,p))
    print X.value
    for i in range(k):
        for j in range(p):
            cur_x = X.value[i,j]
            if cur_x > 0.0000001:
                xs[i,j] = cur_x
    return xs

In [76]:
# import json
raw_vals = [0.8, 0.9, 1.3, 1.9, 0.15]
u = raw_vals / np.sum(raw_vals)
# print json.dumps(list(u))
t = [4000, 4700, 4600, 300, 4800]

p = 20
# u = np.array([0.3, 0.25, 0.45])
# t = np.array([5.0,6.0,2.0])
T = 20000
slot_assignment = mip_assign_slots(p,u,t,T)
print slot_assignment
for i in range(len(slot_assignment)):
    slots_used = np.sum(slot_assignment[i])
    throughput_cap = slots_used*t[i]
    print slots_used, throughput_cap
    

0.636348586294
[[ 0.25        0.25        0.25        0.25        0.25        0.25        0.25
   0.25        0.25        0.25        0.25        0.25        0.25        0.25
   0.25        0.25        0.25        0.25        0.25        0.25      ]
 [ 0.21276596  0.21276596  0.21276596  0.21276596  0.21276596  0.21276596
   0.21276596  0.21276596  0.21276596  0.21276596  0.21276596  0.21276596
   0.21276596  0.21276596  0.21276596  0.21276596  0.21276596  0.21276596
   0.21276596  0.21276596]
 [ 0.2173913   0.2173913   0.2173913   0.2173913   0.2173913   0.2173913
   0.2173913   0.2173913   0.2173913   0.2173913   0.2173913   0.2173913
   0.2173913   0.2173913   0.2173913   0.2173913   0.2173913   0.2173913
   0.2173913   0.2173913 ]
 [ 0.1115094   0.1115094   0.1115094   0.1115094   0.1115094   0.1115094
   0.1115094   0.1115094   0.1115094   0.1115094   0.1115094   0.1115094
   0.1115094   0.1115094   0.1115094   0.1115094   0.1115094   0.1115094
   0.1115094   0.1115094 ]
 [ 0.2083